# Generate QnA synthetic dataset from a Complex PDF using Unstructured

### Overview

We process the PDF by dividing it into three parts.

-   **Text-heavy** - Text-heavy PDF can be processed with open source without the need to use toolkits like Azure AI Document Intelligence or Unstructured.
-   **Image-heavy** - Image-heavy PDF can be converted the entire page to images and let a multimodal LLM like GPT-4o summarize each page.
-   **Mixed** - After reading the document with Azure AI Document Intelligence, we replace the image descriptions inside the figure tags with text summarized by a multimodal LLM. (Often the image descriptions are blank or have only a short caption.)

![summary](../imgs/summary-creating-qna-pdf.png)


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

# aoai_api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
# aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
# aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
# aoai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

# if not aoai_api_version:
#     aoai_api_version = os.getenv("OPENAI_API_VERSION")
# if not aoai_deployment_name:
#     aoai_deployment_name = os.getenv("DEPLOYMENT_NAME")
    
# print(f"aoai_api_endpoint: {aoai_api_endpoint}")
# print(f"aoai_api_key: {aoai_api_key}")
# print(f"aoai_api_version: {aoai_api_version}")
# print(f"aoai_deployment_name: {aoai_deployment_name}")

openai_key = os.getenv('OPENAI_API_KEY')
# print(f"{openai_key=}")

## 1. Read & Preprocess PDF file

---


### Split the PDFs into individual pages


In [5]:
import shutil, random
import openai
from unstructured.cleaners.core import clean_bullets, clean_extra_whitespace, remove_punctuation
from langchain_community.document_loaders import UnstructuredFileLoader, UnstructuredMarkdownLoader, UnstructuredAPIFileLoader
from langchain_community.document_loaders.csv_loader import CSVLoader, UnstructuredCSVLoader
from langchain_ollama import ChatOllama
from util.common_utils import get_language_code

raw_data_dir = "../raw_data"
splitted_raw_data_dir = "splitted_raw_data"
file_path = f"{raw_data_dir}/pdf/vi-vneid.pdf"

DOMAIN = "VNeID Application Usage Guide in Vietnam"
LANGUAGE = "Vietnamese" # You can change your language here. e.g., "Korean", "Japanese", "Chinese"
LANGUAGE_CODE = get_language_code(LANGUAGE)
print(f"Domain: {DOMAIN}, Language: {LANGUAGE}, Language Code: {LANGUAGE_CODE}")

Domain: VNeID Application Usage Guide in Vietnam, Language: Vietnamese, Language Code: vi


(Optional) Only use a poration of the PDF documents for testing. If there are a lot of pages or partial processing is required, cut and save only some pages.


In [6]:
import fitz

# Open the first PDF document
doc1 = fitz.open(file_path)
split_pages = [(7, 15)]

for idx, s in enumerate(split_pages):
    # Create a new empty PDF document
    doc2 = fitz.open()

    # Insert the first 2 pages of doc1 into doc2
    doc2.insert_pdf(doc1, from_page=s[0], to_page=s[1])

    # Save the modified document
    doc2.save(f"{raw_data_dir}/part{idx}.pdf")

In [7]:
from util.common_utils import delete_folder_and_make_folder
from util.preprocess import remove_short_sentences, remove_small_images, analyze_pdf_page_content, split_pdf

file_path = f"{raw_data_dir}/part0.pdf"
analyzed_pdf_result = analyze_pdf_page_content(file_path, text_length_thres=100)
delete_folder_and_make_folder(splitted_raw_data_dir)    
print("### PDF Content Analysis Result:")
for content_type, pages in analyzed_pdf_result.items():
    print(f"{content_type} pages: {pages}")
    split_pdf(file_path, f"{splitted_raw_data_dir}/{content_type}.pdf", pages)

The folder 'splitted_raw_data' and its contents have been deleted.
### PDF Content Analysis Result:
Text pages: [0]
Mixed pages: [1, 2, 3, 5, 6, 7]
Image pages: [4, 8]


### Case 1: Mixed page (Images and text mixed appropriately)

After reading the document with UnstructuredFileLoader, we replace the image descriptions inside the figure tags with text summarized by a multimodal LLM. (Often the image descriptions are blank or have only a short caption.)

- Download tesseract language

```bash
sudo apt-get install tesseract-ocr-eng
sudo apt-get install tesseract-ocr-vie

# Check list lang
tesseract --list-langs
```


In [8]:
%%time

pdf_mixed_path = f"{splitted_raw_data_dir}/Mixed.pdf"

chunk_size = 1500
new_after_n_chars = 1200
combine_text_under_n_chars = 1000
chunk_overlap = 100
max_tokens = 1024
image_dir = "images"

loader = UnstructuredFileLoader(
    file_path=pdf_mixed_path,

    chunking_strategy = "by_title",
    mode="elements",

    extract_image_block_types=["Image", "Table"],
    hi_res_model_name="yolox", #"detectron2_onnx", "yolox", "yolox_quantized"

    extract_images_in_pdf=True,
    skip_infer_table_types='[]', # ['pdf', 'jpg', 'png', 'xls', 'xlsx', 'heic']
    #skip_infer_table_types=True, ## enable to get table as html using tabletrasformer

    extract_image_block_output_dir=image_dir,
    extract_image_block_to_payload=False, ## False: to save image

    max_characters=chunk_size,
    new_after_n_chars=new_after_n_chars,
    combine_text_under_n_chars=combine_text_under_n_chars, # Text less than this number of characters will be combined

    languages= ["vie"],

    post_processors=[clean_bullets, clean_extra_whitespace, remove_punctuation]
)
docs = loader.load()

<timed exec>:10: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
/home/dino/miniconda3/envs/qa-gen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: user 2min 25s, sys: 2.64 s, total: 2min 27s
Wall time: 37.4 s


In [9]:
images = remove_small_images(image_dir, image_dim_thres=16)
tables, texts = [], []

for doc in docs:
    category = doc.metadata["category"]
    if category == "Table": tables.append(doc)
    else: texts.append(doc)

print (f' # texts: {len(texts)} \n # tables: {len(tables)} \n # images: {len(images)}')

 # texts: 3 
 # tables: 0 
 # images: 9


#### Summarize images


In [10]:
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_ollama import ChatOllama
# from langchain_openai import AzureChatOpenAI

# llm = AzureChatOpenAI(
#     temperature=0, 
#     max_tokens=max_tokens,
#     openai_api_version=aoai_api_version,
#     azure_deployment=aoai_deployment_name             
# )
llm = ChatOllama(
    model="gemma3:4b-it-qat",
    temperature=0,
    base_url="http://localhost:11434",
    num_predict=max_tokens
)

system_prompt = "You are an assistant tasked with describing table or image"
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)
human_prompt = [
    {
        "type": "image_url",
        "image_url": {
            "url": "data:image/png;base64," + "{image_base64}",
        },
    },
    {
        "type": "text",
        "text": f"Given image, give a concise summary in {LANGUAGE}. Don't insert any XML tag such as <text> and </text> when answering."
    },
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        system_message_template,
        human_message_template
    ]
)

summarize_chain = prompt | llm | StrOutputParser()
#summarize_chain = {"image_base64": lambda x:x} | prompt | llm_text | StrOutputParser()

In [11]:
%%time
from util.preprocess import encode_image_base64
#images = glob(os.path.join(image_path, "*.jpg"))
base64_images = [encode_image_base64(img_path) for img_path in images]
image_summaries = summarize_chain.batch(base64_images, {"max_concurrency": 3})
image_summaries = remove_short_sentences(image_summaries)

CPU times: user 442 ms, sys: 20 ms, total: 462 ms
Wall time: 23min 2s


In [12]:
image_summaries

# image_summaries = ['Hình ảnh hiển thị một màn hình điện thoại iPhone với các ứng dụng như Dịch thuật, Danh bạ, Phim ảnh, Watch, Mèo, Tiện ích, Năng suất, CGV Cinemas và Số thu chi.',
#  'Hình ảnh hiển thị giao diện ứng dụng liên quan đến phòng chống Covid-19, bao gồm mã QR, thông tin về việc đăng ký, và các biểu tượng khác như tìm kiếm, trò chơi, và thông tin.',
#  'Hình ảnh hiển thị giao diện ứng dụng VNID. Ứng dụng này có chức năng bản đồ, cung cấp thông tin liên hệ và hỗ trợ người dùng. Ứng dụng được đánh giá 2.9 sao, dung lượng 4MB và phù hợp cho độ tuổi từ 3 trở lên.',
#  'Hình ảnh hiển thị ứng dụng VNID, có chức năng quản lý dữ liệu quốc gia và quét mã QR. Ứng dụng này có 9 bài đánh giá, dung lượng 4.0 MB và phù hợp với người dùng từ 3 tuổi trở lên.',
#  'Hình ảnh hiển thị một giao diện tìm kiếm với các dòng tìm kiếm liên quan đến từ "vneid".',
#  'Hình ảnh mô tả quy trình đăng ký và sử dụng bản Cấp có dữ liệu Quốc gia, bao gồm cả việc tích hợp thông tin từ các hệ thống khác nhau và cung cấp thông tin cho người dùng.',
#  'Ảnh chụp màn hình hiển thị giao diện màn hình điện thoại với các ứng dụng như Google, Cửa hàng Google Play, cuộc gọi, cài đặt và ứng dụng hình ảnh.',
#  'Hình ảnh hiển thị giao diện của ứng dụng VNeID, bao gồm các tùy chọn như VNeID, VNeID của bộ công an, và ứng dụng VNeID.',
#  'Hình ảnh hiển thị một màn hình điện thoại với các ứng dụng như Mail, Podcast, App Store và Tiện ích.']

['Hình ảnh hiển thị giao diện ứng dụng liên quan đến phòng chống Covid-19, bao gồm mã QR, thông tin về việc đăng ký, và các biểu tượng khác như tìm kiếm, trò chơi, và thông tin.',
 'Hình ảnh mô tả các thủ tục liên quan đến việc đăng ký và sử dụng hộ tịch, đặc biệt là các thủ tục liên quan đến Cần Cước Dân và việc sử dụng các giấy tờ hộ tịch. Cụ thể, có các thủ tục đăng ký hộ tịch mức 1 và mức 2, cũng như việc chia sẻ thông tin hộ tịch cho các cơ quan chức năng.',
 'Hình ảnh hiển thị giao diện ứng dụng VNID, có chức năng bản đồ và bảo vệ thông tin cá nhân. Ứng dụng này phù hợp cho người dùng từ 3 tuổi trở lên, dung lượng 4MB và đạt đánh giá 2.9 sao.',
 'Hình ảnh hiển thị một bàn phím điện thoại với các chữ cái tiếng Việt và một thanh tìm kiếm hiển thị các từ "vneid" và "vneid".',
 'Hình ảnh hiển thị giao diện của ứng dụng VNeID, bao gồm các tùy chọn như VNeID, VNeID của bộ công an, và ứng dụng VNeID.',
 'Hình ảnh hiển thị màn hình điện thoại với các ứng dụng Google, Cửa hàng Google Play

In [25]:
texts

[Document(metadata={'source': 'splitted_raw_data/Mixed.pdf', 'filetype': 'application/pdf', 'languages': ['vie'], 'last_modified': '2025-04-25T08:57:55', 'page_number': 1, 'orig_elements': 'eJzNWF1v4zYW/SuEn3aB0MPvj0Ff2ulL0V1gFs3uy3QQ8NNSa8tuLM8kW/S/7xUp2c7Y23HTOm0QJNKxLnV5Ds/lpd/9PEvLtEpdf9fG2Ws0k1nKSKjGQUmNhc4Je+UcFvBjBYmRRzq7QbNV6l10vYOYn2dhvb6Pbef6tC33S/e43vV3TWoXTQ8I49xCzAh/bGPfAEqVFIBu1m3XD3Hv3jEu5/oGUSH1nL+/QQfAqgpIq+f2LFBCAJltH7d9Wg1zeds+pOV3GxfS7Bf4IKY+hb5dd3dh6bbbu8392sNjZG6EIAweyO0y9Y+bVGLf/nNWUu4WO7co83o3+9Cm2fuCbvu71Tq2uU2FNUaYxERgJm+JeS31aymH6A1E3nW7lU/3w3yHJPr0MDAyo3OJ3jTf72hKNqDu+x2hhHcLFJpdtxhip0Ru235Z8j9RSgWfmPM4W8mwoJZjIxXBQiWihQhBSn1FpdQcICqpmZOq1AiADgUQwszpWaCE/KpSLynEwDwle/67oomLm2MN/tFu+2+GVM/IEG20OoNNqEoGbBI49pRlbAWjzjsvsrNXlIGW1S+1nQxTAQUkV3/AhTkH1JBnGoaokt9L6oTRJ0r9CFckR7SaTEQulUxzQZTWHiutoLJFmUEyb3EwTMQEskEBvLpzlCBzdewcBYKoqpAqRjkFSsjzJNOEiZeucRj9azeIlFxXVXKxRz9Wg9ndxWopZnKmFvahCAbTUPFMVAknJQmPKUXurlnn5JwA9UDfVOdGQNAKTH46BUrIM9Wiyr68Wp8zGL1UMqlDUj4wnIQBg1mXsL

In [14]:
from util.preprocess import split_text_using_tiktoken

texts_tiktoken = split_text_using_tiktoken(texts, chunk_size, chunk_overlap)

mixed_chunks = image_summaries + texts_tiktoken
print("Length of splits (mixed case): " + str(len(mixed_chunks)))

Length of splits (mixed case): 10


### Case 2: Text-heavy

Text-heavy PDFs can be processed with open source without the need to use toolkits like Azure AI Document Intelligence or Unstructured.


In [15]:
if "Text" in analyzed_pdf_result:

    from langchain_community.document_loaders.pdf import PyMuPDFLoader
    from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

    pdf_text_path = f"{splitted_raw_data_dir}/Text.pdf"
    loader = PyMuPDFLoader(pdf_text_path)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1200, 
        chunk_overlap=200
    )

    text_chunks = text_splitter.split_documents(documents)

    for idx, chunk in enumerate(text_chunks):
        print(f"Chunk {idx}\n{chunk}")
        print("="*80)
        if idx == 2:
            break

    text_chunks = [d.page_content for d in text_chunks]
    print("Length of splits (text-heay case): " + str(len(text_chunks)))
else:
    text_chunks = []

Chunk 0
page_content='7 
 
1. 
GIỚI THIỆU TỔNG QUAN 
1.1 
Đối tượng sử dụng 
- 
Dùng cho công dân Việt Nam có căn cước công dân gắn chíp thực hiện 
đăng ký tài khoản Định danh diện tử 
1.2 
 Mô tả tài liệu 
Nội dung tài liệu bao gồm các phần sau: 
1. 
Mục A: Giới thiệu tổng quan 
2. 
Mục B: Hướng dẫn các chức năng hệ thống có trên APP cho người dân sử 
dụng. 
1.3 
 Thuật ngữ viết tắt 
STT 
Thuật ngữ 
Ý nghĩa 
1 
CCCD 
Căn cước công dân 
2 
SĐT 
Số điện thoại 
3 
NSD 
Người sử dụng 
1.4 Cấu trúc hệ thống 
Sau khi đăng nhập vào hệ thống, màn hình trang chủ hiển thị giao diện như hình.' metadata={'producer': '', 'creator': '', 'creationdate': '', 'source': 'splitted_raw_data/Text.pdf', 'file_path': 'splitted_raw_data/Text.pdf', 'total_pages': 1, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}
Length of splits (text-heay case): 1


### Case 3: Image-heavy

Image-heavy PDF can be converted the entire page to images and let a multimodal LLM like GPT-4o summarize each page.

### Preprocess Image


In [16]:
if "Image" in analyzed_pdf_result:
    import fitz
    from glob import glob

    image_dir = "./pdf_image_tmp"
    delete_folder_and_make_folder(image_dir) 

    pdf_image_path = f"{splitted_raw_data_dir}/Image.pdf"
    doc = fitz.open(pdf_image_path)
    #clip_x, clip_y = 10, 45
    clip_x, clip_y = 10, 10

    for i, page in enumerate(doc):
        x, y, w, h = page.rect
        clip = fitz.Rect(x+clip_x, y+clip_y, w-clip_x, h-clip_y)
        page.set_cropbox(clip)
        pix = page.get_pixmap()
        pix.save(f"{image_dir}/page_{i:03d}.jpg")

    images = sorted(glob(os.path.join(image_dir, "*.jpg")))

The folder './pdf_image_tmp' and its contents have been deleted.


In [17]:
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_openai import AzureChatOpenAI

max_tokens = 1024
# llm = AzureChatOpenAI(
#     temperature=0, 
#     max_tokens=max_tokens,
#     openai_api_version=aoai_api_version,
#     azure_deployment=aoai_deployment_name                  
# )
llm = ChatOllama(
    model="gemma3:4b-it-qat",
    temperature=0,
    base_url="http://localhost:11434",
    num_predict=max_tokens
)

human_prompt_main = f"Given image, give a concise summary in {LANGUAGE}. Don't insert any XML tag such as <text> and </text> when answering."

system_prompt = "You are an assistant tasked with describing table or image, specialized in Smartphone product."
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)
human_prompt = [
    {
        "type": "image_url",
        "image_url": {
            "url": "data:image/png;base64," + "{image_base64}",
        },
    },
    {
        "type": "text",
        "text": human_prompt_main
    },
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        system_message_template,
        human_message_template
    ]
)

summarize_chain = prompt | llm | StrOutputParser()

In [18]:
%%time
if "Image" in analyzed_pdf_result:
    from util.preprocess import encode_image_base64
    #images = glob(os.path.join(image_path, "*.jpg"))
    base64_images = [encode_image_base64(img_path) for img_path in images]
    image_summaries = summarize_chain.batch(base64_images, {"max_concurrency": 8})
    image_summaries = remove_short_sentences(image_summaries)
    print("Length of image_summaries (image-heavy case): " + str(len(image_summaries)))
else:
    image_summaries = []


Length of image_summaries (image-heavy case): 2
CPU times: user 88.4 ms, sys: 5.15 ms, total: 93.5 ms
Wall time: 5min 29s


In [19]:
image_summaries

['Hình ảnh này hướng dẫn sử dụng mã QR để đăng nhập vào các ứng dụng trên điện thoại, bao gồm cả các ứng dụng chính phủ và các ứng dụng khác.',
 'Hình ảnh minh họa giao diện ứng dụng định danh điện tử quốc gia Việt Nam (VNeID) với các biểu tượng liên quan đến xác thực và đăng nhập.']

## 2. Construct QnA Pairs

---

### Option 1.

Leverage the `azure-ai-generative` package. The `QADataGenerator` class in this package makes it easy to generate QnA synthetic questions. However, using this class as is has the disadvantage of not being able to use custom prompts, so we inherited from it and created the `CustomQADataGenerator` class.


### Option 2.

You write the entire sequence of code to create a QnA dataset without using a separate toolkit.


In [ ]:
# aoai_api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
# aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
# aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
# aoai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

In [22]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import JsonOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from util.qa_pair import get_qna_prompt_template, QAPair

# llm = AzureChatOpenAI(
#     temperature=0, 
#     max_tokens=1024,
#     openai_api_version=aoai_api_version,
#     azure_deployment=aoai_deployment_name                    
# )

llm = ChatOllama(
    model="gemma3:4b-it-qat",
    temperature=0,
    base_url="http://localhost:11434",
    num_predict=max_tokens
)

parser = JsonOutputParser(pydantic_object=QAPair)
prompt = get_qna_prompt_template(LANGUAGE)

chain = prompt | llm | parser

In [23]:
input_batch = []

for doc in mixed_chunks:
    dic = {"context": doc, "domain": DOMAIN, "num_questions": "3"}
    input_batch.append(dic)

for doc in text_chunks:
    dic = {"context": doc, "domain": DOMAIN, "num_questions": "3"}
    input_batch.append(dic)

for doc in image_summaries:
    dic = {"context": doc, "domain": DOMAIN, "num_questions": "3"}
    input_batch.append(dic)        

In [24]:
%%time
qa_pair = chain.batch(input_batch, {"max_concurrency": 5})

CPU times: user 2.03 s, sys: 73.2 ms, total: 2.11 s
Wall time: 18min 40s


In [26]:
qa_pair

[[{'QUESTION': 'Ứng dụng VNeID hiển thị mã QR để làm gì?',
   'ANSWER': 'Ứng dụng VNeID hiển thị mã QR để đăng ký thông tin liên quan đến phòng chống Covid-19.'},
  {'QUESTION': 'Giao diện ứng dụng VNeID có những biểu tượng nào?',
   'ANSWER': 'Giao diện ứng dụng VNeID bao gồm các biểu tượng tìm kiếm, trò chơi, và thông tin.'},
  {'QUESTION': 'Ứng dụng VNeID có liên quan đến việc đăng ký thông tin gì?',
   'ANSWER': 'Ứng dụng VNeID liên quan đến việc đăng ký thông tin liên quan đến phòng chống Covid-19.'}],
 [{'QUESTION': 'Theo thông tin được cung cấp, các thủ tục liên quan đến hộ tịch bao gồm những loại thủ tục nào?',
   'ANSWER': 'Thông tin cho thấy các thủ tục liên quan đến hộ tịch bao gồm đăng ký hộ tịch mức 1 và mức 2, cũng như việc chia sẻ thông tin hộ tịch cho các cơ quan chức năng.'},
  {'QUESTION': 'Hình ảnh mô tả những thủ tục nào liên quan đến việc sử dụng giấy tờ hộ tịch?',
   'ANSWER': 'Hình ảnh mô tả các thủ tục liên quan đến việc sử dụng hộ tịch, đặc biệt là các thủ tục 

## 3. Save to jsonl

---

If you want to augment dataset, you can try Evovle-Instruct or other data augmentation techniques.<br>
Please refer to `../evolve-instruct` and `../glan-instruct` for more details.


In [28]:
import json
from util.common_utils import convert_to_oai_format, save_jsonl

output_dir = './dataset'
os.makedirs(output_dir, exist_ok=True)

system_prompt_msg = f"""You are the SME (Subject Matter Expert) in {DOMAIN}. Please answer the questions accurately. If the question is in {LANGUAGE}, write your answer in {LANGUAGE}."""

save_filename = "advertising"
oai_qa_pair = convert_to_oai_format(qa_pair, system_prompt_msg=system_prompt_msg)

#save_jsonl(qa_pair, f"{output_dir}/{save_filename}.jsonl")
save_jsonl(oai_qa_pair, f"{output_dir}/{save_filename}-oai.jsonl")

### Clean up


In [ ]:
!rm -rf {splitted_raw_data_dir} pdf_image_tmp pdf_mixed_tmp outputs_tmp images